In [48]:
import sys
sys.path.append("/Users/aaron/phd/impact-of-contamination-on-taxonomy")

In [49]:
from workflow.external.gtdb_metadata import GtdbMetadataR207

DF_META = GtdbMetadataR207().output().read_cached()
DF_META.head()

,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,contig_count,...,gtdb_type_species_of_genus,ncbi_taxonomy,ncbi_taxonomy_unfiltered,domain,phylum,class,order,family,genus,species
gid,,,,,,,,,,,,,,,,,,,,,
GCA_000006155.2,1916,93.12,0.0,1171,g__Bacillus (UID902),324,0.0,4305660,80.178992,426,...,f,d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,d__Bacteria;x__Terrabacteria group;p__Firmicut...,d__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae_G,g__Bacillus_A,s__Bacillus_A anthracis
GCA_000007325.1,1,99.95,0.0,149,k__Bacteria (UID2329),89,0.0,1973459,90.754610,1,...,t,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria,p__Fusobacteriota,c__Fusobacteriia,o__Fusobacteriales,f__Fusobacteriaceae,g__Fusobacterium,s__Fusobacterium nucleatum
GCA_000007385.1,0,99.82,0.0,481,c__Gammaproteobacteria (UID4202),276,0.0,4190634,84.805944,1,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Xanthomonadales,f__Xanthomonadaceae,g__Xanthomonas,s__Xanthomonas oryzae
GCA_000008085.1,0,73.13,0.0,149,k__Archaea (UID2),107,0.0,462902,94.299480,1,...,t,d__Archaea;p__Nanoarchaeota;c__;o__Nanoarchaea...,d__Archaea;x__DPANN group;p__Nanoarchaeota;o__...,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanoarchaeaceae,g__Nanoarchaeum,s__Nanoarchaeum equitans
GCA_000008885.1,0,100.00,0.0,134,k__Bacteria (UID2495),80,0.0,617456,87.831079,2,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales_A,f__Enterobacteriaceae_A,g__Wigglesworthia,s__Wigglesworthia glossinidia_A


In [50]:
from workflow.fasttree_marker_split.f_a_compare_patristic_distance import FastTreeMarkerSplitComparePatristicDistance

DF = FastTreeMarkerSplitComparePatristicDistance().output().maybe_read_cached()
DF.head()

FileNotFoundError: File /srv/home/uqamussi/projects/gunc-chimeras/output/fasttree_marker_split/pct_50/patristic_distances.h5 does not exist

In [ ]:
DF.sort_values(by='pd_between_halves', ascending=False)

In [ ]:
from workflow.fasttree_marker_split.e_decorate_fasttree import FastTreeMarkerSplitDecorateFastTree
import os


# Load the taxonomy string from the decorated tree
def get_gid_tax_string():
    out = dict()
    path = FastTreeMarkerSplitDecorateFastTree().output().path
    path = os.path.join(os.path.dirname(path), 'decorated.tree-taxonomy')
    with open(path) as f:
        for line in f.readlines():
            gid, tax = line.strip().split('\t')
            out[gid] = tax
    return out

D_GID_TO_SPLIT_TAX = get_gid_tax_string()
print(list(D_GID_TO_SPLIT_TAX.items())[0:3])

In [ ]:
from tqdm import tqdm
from collections import defaultdict, Counter


def get_taxonomic_novelty_v2(df):

    # Get the number of taxa contained within each rank
    d_tax_under = defaultdict(lambda: 0)
    for taxonomy in df['gtdb_taxonomy'].values:
        for rank in taxonomy.split(';'):
            d_tax_under[rank] += 1

    # Find the novelty
    n_total = len(df)

    # Process the rows
    d_gid_to_tax_novelty = dict()
    for gid, row in tqdm(df.iterrows(), total=n_total):
        tax_str = row['gtdb_taxonomy']
        taxonomy = tax_str.split(';')
        last_rank = 'st'
        for taxon in reversed(taxonomy):
            taxon_count = d_tax_under[taxon]
            if taxon_count > 1:
                break
            last_rank = taxon[0]
        d_gid_to_tax_novelty[gid] = D_RANK[last_rank]

    return d_gid_to_tax_novelty


D_GID_TO_TAX_NOVELTY = get_taxonomic_novelty_v2(DF_META)



In [ ]:
from workflow.util.tree import parse_label
import dendropy


def get_first_named_node_in_tree():
    tree: dendropy.Tree = FastTreeMarkerSplitDecorateFastTree().output().read()

    out = dict()
    for leaf_node in tqdm(tree.leaf_node_iter(), total=len(tree.taxon_namespace)):
        label = leaf_node.taxon.label
        gid = label.replace('TEST_', '')
        if label.startswith('TEST_'):
            cur_node = leaf_node.parent_node
            while cur_node is not None:
                support, taxon, auxiliary_info = parse_label(cur_node.label)
                if taxon is not None:
                    out[gid] = taxon
                    break
                cur_node = cur_node.parent_node
    return out

D_GID_TO_FIRST_NAMED_NODE = get_first_named_node_in_tree()
print(list(D_GID_TO_FIRST_NAMED_NODE.items())[0:10])

In [72]:
from collections import defaultdict

D_RANK = {'d': 'domain',
          'p': 'phylum',
          'c': 'class',
          'o': 'order',
          'f': 'family',
          'g': 'genus',
          's': 'species',
          'st': 'strain'}

D_NOVELTY_TO_PARENT_TAXON_PREFIX = {
    'phylum': 'd',
    'class': 'p',
    'order': 'c',
    'family': 'o',
    'genus': 'f',
    'species': 'g',
    'strain': 's',
}

def parse_phylorank_tax_string(taxonomy):
    out = defaultdict(list)
    for rank in taxonomy.split('; '):
        out[rank[0]].append(rank)
    return out

def split_to_full_string(split_string, prev_rank):
    out = list()
    seen = False
    for rank, lst_tax in split_string.items():
        if rank == prev_rank:
            seen = True
        if seen:
            out.extend(lst_tax)
    return ';'.join(out)



def get_rank_agree_for_fail_gids():
    out = dict()
    out_short = dict()
    for gid in sorted(set(DF['gid'])):
        true_tax = DF_META.loc[gid]['gtdb_taxonomy']
        true_tax_parsed = {x[0]: x for x in true_tax.split(';')}
        tax_novelty = D_GID_TO_TAX_NOVELTY[gid]

        if gid == 'GCA_002708525.1':
            print()

        for suffix in ('', '_C'):
            gid_suffix = f'{gid}{suffix}'
            split_tax = D_GID_TO_SPLIT_TAX[f'TEST_{gid_suffix}']
            split_tax_parsed = parse_phylorank_tax_string(split_tax)

            # Check to see if the higher rank (above the novel rank is correct)
            rank_prefix_tax_novelty = D_NOVELTY_TO_PARENT_TAXON_PREFIX[tax_novelty]
            rank_lst = list(reversed(D_NOVELTY_TO_PARENT_TAXON_PREFIX.values()))
            for rank_prefix in rank_lst[rank_lst.index(rank_prefix_tax_novelty):]:

                split_tax_at_novel_parent = split_tax_parsed[rank_prefix]
                true_tax_at_novel_parent = true_tax_parsed[rank_prefix]

                if len(split_tax_at_novel_parent) == 1:
                    cur_tax = split_tax_at_novel_parent[0]

                    # This matches, check to make sure the lower ranks agree too
                    if cur_tax == true_tax_at_novel_parent:
                        break

                    else:
                        print('?')

                else:
                    print('?')


            #
            # first_named_node = D_GID_TO_FIRST_NAMED_NODE[gid_suffix]
            # first_named_node_set = set(first_named_node.split('; '))
            # rank = first_named_node[0]
            #
            # expected_named_node = split_tax_parsed[rank]
            # expected_named_node_set = set(expected_named_node)
            #
            # if len(expected_named_node) == 1:
            #     if len(expected_named_node_set.intersection(first_named_node_set)) > 0:
            #         out[gid_suffix] = D_RANK[rank]
            #         out_short[gid_suffix] = split_to_full_string(split_tax_parsed, rank)
            #     else:
            #         print('?')
            # else:
            #     if len(expected_named_node_set.intersection(first_named_node_set)) > 0:
            #         out[gid_suffix] = D_RANK[rank]
            #         out_short[gid_suffix] = split_to_full_string(split_tax_parsed, rank)
            #     else:
            #         print('??')

            #
            # if gid in {'GCA_012797725.1', 'GCA_013334845.1', 'GCA_002708525.1'}:
            #     print('asd')
            #
            # prev_rank = 's'
            # for rank, lst_tax in reversed(split_tax_parsed.items()):
            #     if len(lst_tax) == 1:
            #         if lst_tax[0] != true_tax_parsed[rank] and len(lst_tax[0]) > 3:
            #             out[gid_suffix] = D_RANK[prev_rank]
            #             out_short[gid_suffix] = split_to_full_string(split_tax_parsed, rank)
            #             break
            #     else:
            #         out[gid_suffix] = D_RANK[prev_rank]
            #         out_short[gid_suffix] = split_to_full_string(split_tax_parsed, rank)
            #         break
            #     prev_rank = rank
            # else:
            #     out[gid_suffix] = D_RANK[prev_rank]
            #     out_short[gid_suffix] = ''
    return out, out_short




D_GID_TO_RANK_AGREE, D_GID_TO_SHORT_TAX = get_rank_agree_for_fail_gids()
print(list(D_GID_TO_RANK_AGREE.items())[0:50])
print(list(D_GID_TO_SHORT_TAX.items())[0:50])

?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?
?

?
?
?


KeyboardInterrupt: 

In [62]:
df_tmp = DF[['gid', 'pd_between_halves']].copy()
df_tmp['pd_between_halves'] = df_tmp['pd_between_halves'].apply(lambda x: x * 100000000)

tax_values = list()
novelty_vals = list()
chim_agree_rank = list()
keep_agree_rank = list()
split_tax_values_keep = list()
split_tax_values_chim = list()
for row in df_tmp.itertuples():
    tax = DF_META.loc[row.gid]['gtdb_taxonomy']
    tax_values.append(tax)
    novelty_vals.append(D_GID_TO_TAX_NOVELTY[row.gid])
    keep_agree_rank.append(D_GID_TO_RANK_AGREE[row.gid])
    chim_agree_rank.append(D_GID_TO_RANK_AGREE[f'{row.gid}_C'])
    split_tax_values_keep.append(D_GID_TO_SHORT_TAX[row.gid])
    split_tax_values_chim.append(D_GID_TO_SHORT_TAX[f'{row.gid}_C'])

df_tmp['novelty'] = novelty_vals
df_tmp['keep_highest_agree'] = keep_agree_rank
df_tmp['chim_highest_agree'] = chim_agree_rank
df_tmp['taxonomy_r207'] = tax_values
df_tmp['taxonomy_keep_half'] = split_tax_values_keep
df_tmp['taxonomy_chim_half'] = split_tax_values_chim

df_tmp.sort_values(by=['pd_between_halves'], ascending=[False], inplace=True)
df_tmp.to_csv('/tmp/pd_halves.tsv', sep='\t', index=False)

df_tmp.head()

,gid,pd_between_halves,novelty,keep_highest_agree,chim_highest_agree,taxonomy_r207,taxonomy_keep_half,taxonomy_chim_half
16347,GCA_002708525.1,36781.708962,species,genus,class,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Pelagibacter;s__Pelagibacter sp002708525,c__Alphaproteobacteria;o__;f__;g__;s__
21940,GCA_016203935.1,24823.760765,order,phylum,class,d__Bacteria;p__Patescibacteria;c__Andersenbact...,p__Patescibacteria;c__;o__;f__;g__;s__,c__ABY1;o__JACQLU01;f__JACQLU01;g__JACQLU01;s_...
12651,GCA_002171275.2,23230.821790,species,genus,species,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__GCA-2704625;s__GCA-2704625 sp002171275,s__GCA-002690875 sp002690875
23830,GCA_014385135.2,22857.132028,strain,species,species,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,s__Merdisoma sp014290175,s__Pseudoruminococcus_A massiliensis
5722,GCA_003524845.1,21517.597590,genus,class,family,d__Bacteria;p__Patescibacteria;c__Microgenomat...,c__Microgenomatia;o__;f__;g__;s__,f__UBA10105;g__UBA10118;s__UBA10118 sp003524845


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(10, 5), sharey=True, sharex=True)

ax_st = axes[0, 0]
ax_sp = axes[0, 1]
ax_ge = axes[0, 2]
ax_fa = axes[1, 0]
ax_or = axes[1, 1]
ax_class = axes[1, 2]

ax_st.title.set_text('Strain')
ax_sp.title.set_text('Species')
ax_ge.title.set_text('Genus')
ax_fa.title.set_text('Family')
ax_or.title.set_text('Order')
ax_class.title.set_text('Class')

n_bins = 100

sns.histplot(df_tmp[df_tmp['novelty'] == 'strain'], x='pd_between_halves', stat='percent', ax=ax_st, bins=n_bins)
sns.histplot(df_tmp[df_tmp['novelty'] == 'species'], x='pd_between_halves', stat='percent', ax=ax_sp, bins=n_bins)
sns.histplot(df_tmp[df_tmp['novelty'] == 'genus'], x='pd_between_halves', stat='percent', ax=ax_ge, bins=n_bins)
sns.histplot(df_tmp[df_tmp['novelty'] == 'family'], x='pd_between_halves', stat='percent', ax=ax_fa, bins=n_bins)
sns.histplot(df_tmp[df_tmp['novelty'] == 'order'], x='pd_between_halves', stat='percent', ax=ax_or, bins=n_bins)
sns.histplot(df_tmp[df_tmp['novelty'] == 'class'], x='pd_between_halves', stat='percent', ax=ax_class, bins=n_bins)
plt.show()